In [7]:
import unicodedata
import collections
import random
import warnings

from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path


In [8]:
#warnings.filterwarnings("ignore", "FutureWarning")
get_ipython().run_line_magic('matplotlib', 'inline')


In [9]:
# Глобальные константы
# Символы
RUS_LETTERS = 'АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя'
NUMBERS = '0123456789'
PUNCTUATION_MARKS = ' .,?:;—!<>-«»()[]*"'
ALL_SYMBOLS = RUS_LETTERS + NUMBERS + PUNCTUATION_MARKS

# Пути
raw_data_path ='raw_data/russian_news_corpus/'
source_file = 'russian_news.txt'
save_label_path = 'raw_data/synth_dataset/'
save_img_path = 'raw_data/synth_dataset/images/'

# Шрифты
font = ImageFont.truetype('raw_data/fonts/Roboto-Regular.ttf', 20)

# Ширина генерируемых картинок
width =360
# Количество генерируемых картинок в датасете
img_amount = 20000



In [10]:
# Максимальное кол-ва символов, вмещающихся в заданную ширину картинки 
def get_max_i(width,font,symbols=ALL_SYMBOLS)-> int:
    max_w_char = 0
    for char in symbols:
        if font.getlength(char) > max_w_char:
            max_w_char = font.getlength(char)
    max_i = round(width/max_w_char)-1
    return max_i


# (Минимально) допустимая высота картинки 
def get_height(font,symbols=ALL_SYMBOLS) -> int:
    return font.getbbox(symbols)[3]+6


# Проверка на распознаваемость:
# Критерии:
# 1: все символы из набора symbols
# 2: есть хотя бы одна буква/цифра
# 3: есть хотя бы два непробельных символа
def is_applicable(text,symbols=ALL_SYMBOLS) -> bool:
    cond1 = all(element in symbols for element in text)
    cond2 = any(element in (RUS_LETTERS+NUMBERS) for element in text)
    cond3 = len(text) > 1
    return cond1 and cond2

In [11]:
max_i = get_max_i(width,font,ALL_SYMBOLS)
height = get_height(font,ALL_SYMBOLS)
print(max_i, height)

18 32


In [13]:
labels = [] 
i = 0
filler = ' '*max_i
with open(raw_data_path+source_file, 'r', encoding="utf-8") as f:
    while i< img_amount :
        line = f.read(max_i)
        # Возьмем целые слова из прочитанной строки
        first_space = line.find(' ')
        last_space = line.rfind(' ')
        line = line[first_space:last_space]
        
        # Проверка на отсутствие сторонних символов
        if is_applicable(line):
            i+=1
            line = (line + filler)[0:max_i]
            w = random.randint(200, 255)
            b = random.randint(0, 100)
        
            line = line.upper()
            # созадим картинку: символы в строке в один ряд
            img = Image.new('L', (width, height), color = (w, ))
            drawing = ImageDraw.Draw(img)
            drawing.text((5, 5), line, font=font, fill=(b,))
            img_name = f'{i}_pasp.png'
            img.save(save_img_path + img_name)
            labels.append(img_name + ' : '+ line)
            
#print (labels [0])
            
with open(save_label_path + 'labels.txt', 'w') as f:
    for element in labels:
        f.write(element)
        f.write('\n')